In [8]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
import os
import numpy as np
import pandas as pd
seed=0
np.random.seed(seed)

from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights, load_qmodel
import hls4ml

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [9]:
df = pd.read_csv('./breast-cancer-wisconsin.csv', sep = ';')
X = df.drop('Y', axis = 1).values
y = df.Y

In [10]:
encoder = OneHotEncoder()
Y_reshaped = [[label] for label in y]
Y_encoded = encoder.fit_transform(Y_reshaped)

In [11]:
Y_encoded = Y_encoded.toarray()

In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y_encoded,test_size=0.3,random_state=42)

In [13]:
sc = MinMaxScaler(feature_range=(0,0.9))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Train for different precision

In [22]:
model_num = 20
id_num = 0
max_RF = 100
precision = 8
dirname = './models/models_' + str(id_num) + "/prec_" + str(precision) + "/model_"
dirname

In [15]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights, load_qmodel
import hls4ml



In [17]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
import qkeras.qtools.qtools_util
import qkeras.estimate
loaded_models = []
for model_no in range(model_num):
    co = {}
    _add_supported_quantized_objects(co)
    loaded_models.append(load_model(dirname + str(model_no) + '/KERAS_check_best_model.h5', custom_objects=co))
#model = 


In [23]:
ready = True
models = []
first_layer = []
sec_layer = []
model_num = 20
model_layers = [[] for x in range(model_num)]
bits = 8
int_bits = 0
for i in range(model_num):
    if ready:
        input1 = 10
        layer1 = random.randint(5,20)                           ###64
        first_layer.append(layer1)
        layer2 = random.randint(5,20)                           ###32
        sec_layer.append(layer2)
        output1 = 2
        model_layers[i].append(layer1)
        model_layers[i].append(layer2)
        if not (os.path.exists('layerdetails/models_' + str(id_num) + '/model_' + str(i))):
            os.makedirs('layerdetails/models_' + str(id_num) + '/model_' + str(i))
        with open('layerdetails/models_' + str(id_num) + '/model_' + str(i) + "/layers.txt",'w') as f:
            f.write(str(layer1) + " " + str(layer2))
        f.close
        #np.savetxt(dirname + str(i) + '/layerdetails.txt',model_layers[i],delimiter="\n" )
    else:
        with open('layerdetails/models_' + str(id_num) + '/model_' + str(i) + "/layers.txt",'r') as r:
            #print(r.readlines()[0])
            layer = r.readlines()[0].split(" ")
            input1 = 10
            layer1 = int(layer[0])
            layer2 = int(layer[1])
            output1 = 2
            first_layer.append(layer1)
            sec_layer.append(layer2)
    model=Sequential()
    model.add(QDense(layer1, input_shape=(input1,), name='fc1', kernel_quantizer=quantized_bits(bits,int_bits,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(bits,int_bits,alpha=1),
                    kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
    model.add(QActivation(activation=quantized_relu(bits,int_bits,use_stochastic_rounding=False), name='relu1'))
    model.add(QDense(layer2, name='fc2',
                    kernel_quantizer=quantized_bits(bits,int_bits,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(bits,int_bits,alpha=1),
                    kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
    model.add(QActivation(activation=quantized_relu(bits,int_bits,use_stochastic_rounding=False), name='relu2'))
    model.add(QDense(output1, name='output',
                    kernel_quantizer=quantized_bits(bits,int_bits,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(bits,int_bits,alpha=1),
                    kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
    model.add(Activation(activation='softmax', name='softmax'))
    models.append(model)
    
    
    
    
    
model_layers = np.array(model_layers)

In [20]:
%%time
Train = True
selector = 1
for precision in range(8,1,-1):
    dirname = './models/models_' + str(id_num) + "/prec_" + str(precision) + "/model_"
    for i in range(0,model_num,1):    
        adam = Adam(lr=0.02)
        if(Train):
            loaded_models = models
            pruning_params = {"pruning_schedule" : pruning_schedule.ConstantSparsity(0.2, begin_step=2000, frequency=100)}
            models[i] = prune.prune_low_magnitude(models[i], **pruning_params)
            models[i].compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
            callbacks= all_callbacks( outputDir = 'arrythmia_classification_prune_new')
            callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())

    #         models[i].fit(X_train, Y_train, batch_size=1,
    #                   epochs=15,validation_split=0.2, verbose=1, shuffle=True,
    #                   callbacks = callbacks.callbacks);
            models[i].fit(X_train,Y_train,batch_size=64,epochs=10,
                validation_split=0.25,
                verbose=1,
                shuffle=True,
                callbacks=callbacks.callbacks,
                )
            models[i] = strip_pruning(models[i])
            #model_save_quantized_weights(models[i], "test_weights_new_" + str(i) + "_" + str(id_num))
            models[i].save(dirname + str(i) + '/KERAS_check_best_model.h5')
        rf_range = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]
        for j in rf_range:
            loaded_models[i].save(dirname + str(i) + '/KERAS_check_best_model.h5')
            config = hls4ml.utils.config_from_keras_model(loaded_models[i],granularity='name')
            #config['Model']['Precision']='fixed<16,4>'
            #config['LayerName']['fc1_input']['Precision']['result'] = 'fixed<8,2>'
            #config['LayerName']['output']['Precision']['result'] = 'fixed<16,4>'
            config['LayerName']['fc1_input']['Precision']['result'] = 'fixed<' + str(precision) + ',2>'
            config['LayerName']['relu1']['Precision']['result'] = 'ap_ufixed<' + str(precision) + ',0,AP_RND_CONV,AP_SAT>'
            config['LayerName']['relu2']['Precision']['result'] = 'ap_ufixed<' + str(precision) + ',0,AP_RND_CONV,AP_SAT>'

            config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
            config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
            config['Model']['ReuseFactor'] = j
            #config['LayerName']['fc1']['Precision']['result'] = 'fixed<16,4>'
            #config['LayerName']['fc2']['Precision']['result'] = 'fixed<16,4>'

            hls_model = hls4ml.converters.convert_from_keras_model(
                loaded_models[i],hls_config = config,output_dir = dirname + str(i) + '/reuse_' + str(j + 1) + '/hls4ml_prj',part = 'xc7z007s-clg225-2')
            hls_model.compile()
            os.system('cp -r build_prj_orig.tcl project_orig.tcl build_prj_orig_lut.tcl ' + dirname + str(i) + '/reuse_' + str(j + 1) + '/hls4ml_prj')
            os.system('cp init.sh init_lut.sh ' + dirname + str(i) + '/reuse_' + str(j + 1) + '/hls4ml_prj')
            start_dir = os.getcwd()
            os.chdir(r'./models/models_'+ str(id_num) + "/prec_" + str(precision) + "/model_" + str(i) + '/reuse_' + str(j + 1) + '/hls4ml_prj')
            if selector:
                os.system('bash init.sh')
                os.system('rm init_lut.sh')
            else:
                os.system('rm init.sh')
                os.system('bash init_lut.sh')
            os.chdir(start_dir)        
            if j==9:
                     for num in range(j+2):
                         os.system('rm -r ' + dirname +str(i) + '/reuse_' + str(num) + '/hls4ml_prj/myproject_prj/solution1/.autopilot')
